# Reinforcement Learning: From Fundamentals to Deep RL

**Complete Guide to Reinforcement Learning**

Learn RL from first principles to modern deep RL algorithms like DQN, Policy Gradients, and PPO.

---

## Table of Contents

1. [Introduction to Reinforcement Learning](#1-introduction)
2. [Markov Decision Processes (MDPs)](#2-markov-decision-processes)
3. [Classical RL: Q-Learning](#3-classical-rl-q-learning)
4. [Deep Q-Networks (DQN)](#4-deep-q-networks-dqn)
5. [Policy Gradient Methods](#5-policy-gradient-methods)
6. [Actor-Critic Methods](#6-actor-critic-methods)
7. [Proximal Policy Optimization (PPO)](#7-proximal-policy-optimization)
8. [Advanced Topics](#8-advanced-topics)
9. [Interview Questions](#9-interview-questions)

---

In [ ]:
# Install required packages
# !pip install gymnasium torch numpy matplotlib imageio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque, namedtuple
import random
import gymnasium as gym

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Matplotlib settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

---

## 1. Introduction to Reinforcement Learning

### What is Reinforcement Learning?

**Reinforcement Learning (RL)** is learning by interaction with an environment to maximize cumulative reward.

**Key Concepts:**
- **Agent**: The learner/decision maker
- **Environment**: What the agent interacts with
- **State** ($s_t$): Current situation of the agent
- **Action** ($a_t$): Decision made by agent
- **Reward** ($r_t$): Feedback from environment
- **Policy** ($\pi$): Strategy for choosing actions

**RL vs Other ML:**
- **Supervised Learning**: Learn from labeled examples
- **Unsupervised Learning**: Find patterns in data
- **Reinforcement Learning**: Learn from rewards through trial and error

### The RL Framework

```
Agent                    Environment
  |                           |
  |  ------ action a_t ---->  |
  |                           |
  |  <--- state s_t+1 ------  |
  |  <--- reward r_t+1 -----  |
```

**Goal**: Learn policy $\pi$ that maximizes expected cumulative reward:

$$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ... = \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}$$

where $\gamma \in [0, 1]$ is the discount factor.

### Simple Grid World Example

In [ ]:
class SimpleGridWorld:
    """
    Simple 4x4 grid world environment
    
    Agent starts at (0, 0) and tries to reach goal at (3, 3)
    Actions: up, down, left, right
    Reward: -1 per step, +10 at goal
    """
    
    def __init__(self, size=4):
        self.size = size
        self.reset()
        
    def reset(self):
        """Reset to start state"""
        self.agent_pos = [0, 0]
        self.goal_pos = [self.size - 1, self.size - 1]
        return tuple(self.agent_pos)
    
    def step(self, action):
        """
        Take action and return (next_state, reward, done)
        
        Actions: 0=up, 1=down, 2=left, 3=right
        """
        # Update position based on action
        if action == 0:  # up
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:  # down
            self.agent_pos[0] = min(self.size - 1, self.agent_pos[0] + 1)
        elif action == 2:  # left
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        elif action == 3:  # right
            self.agent_pos[1] = min(self.size - 1, self.agent_pos[1] + 1)
        
        # Compute reward
        if self.agent_pos == self.goal_pos:
            reward = 10.0
            done = True
        else:
            reward = -1.0
            done = False
        
        return tuple(self.agent_pos), reward, done
    
    def render(self):
        """Visualize grid world"""
        grid = np.zeros((self.size, self.size))
        grid[self.agent_pos[0], self.agent_pos[1]] = 1  # Agent
        grid[self.goal_pos[0], self.goal_pos[1]] = 2    # Goal
        
        plt.figure(figsize=(4, 4))
        plt.imshow(grid, cmap='viridis', vmin=0, vmax=2)
        plt.colorbar(ticks=[0, 1, 2], label='0=Empty, 1=Agent, 2=Goal')
        plt.title('Grid World')
        plt.grid(True)
        plt.show()

# Test the environment
env = SimpleGridWorld(size=4)
print("Initial state:", env.reset())
env.render()

# Take some random actions
print("\nTaking random actions:")
for i in range(5):
    action = np.random.randint(0, 4)
    state, reward, done = env.step(action)
    print(f"Action: {action}, State: {state}, Reward: {reward}, Done: {done}")
    if done:
        break

---

## 2. Markov Decision Processes (MDPs)

### Mathematical Framework

An MDP is defined by tuple $(S, A, P, R, \gamma)$:
- $S$: Set of states
- $A$: Set of actions
- $P(s'|s,a)$: Transition probability
- $R(s,a,s')$: Reward function
- $\gamma$: Discount factor

**Markov Property**: Future depends only on current state, not history:

$$P(s_{t+1}|s_t, a_t, s_{t-1}, a_{t-1}, ...) = P(s_{t+1}|s_t, a_t)$$

### Key Functions

**State-Value Function** $V^\pi(s)$: Expected return starting from state $s$ under policy $\pi$

$$V^\pi(s) = E_\pi[G_t | s_t = s] = E_\pi\left[\sum_{k=0}^{\infty} \gamma^k r_{t+k+1} \bigg| s_t = s\right]$$

**Action-Value Function** $Q^\pi(s,a)$: Expected return starting from state $s$, taking action $a$, then following policy $\pi$

$$Q^\pi(s,a) = E_\pi[G_t | s_t = s, a_t = a]$$

### Bellman Equations

**Bellman Expectation Equation**:

$$V^\pi(s) = \sum_a \pi(a|s) \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^\pi(s')]$$

$$Q^\pi(s,a) = \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma \sum_{a'} \pi(a'|s') Q^\pi(s',a')]$$

**Bellman Optimality Equation**:

$$V^*(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^*(s')]$$

$$Q^*(s,a) = \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma \max_{a'} Q^*(s',a')]$$

### Value Iteration Example

In [ ]:
class ValueIteration:
    """
    Value Iteration algorithm for solving MDPs
    
    Iteratively updates value function using Bellman optimality equation
    until convergence.
    """
    
    def __init__(self, env, gamma=0.99, theta=1e-6):
        self.env = env
        self.gamma = gamma  # Discount factor
        self.theta = theta  # Convergence threshold
        
        # Initialize value function
        self.V = np.zeros((env.size, env.size))
        
    def solve(self, max_iterations=1000):
        """
        Run value iteration until convergence
        """
        for iteration in range(max_iterations):
            delta = 0
            
            # Update value for each state
            for i in range(self.env.size):
                for j in range(self.env.size):
                    if [i, j] == self.env.goal_pos:
                        continue  # Goal state has value 0
                    
                    v = self.V[i, j]
                    
                    # Compute max over actions
                    action_values = []
                    for action in range(4):  # 4 actions
                        # Simulate taking action
                        self.env.agent_pos = [i, j]
                        next_state, reward, done = self.env.step(action)
                        
                        # Bellman update
                        value = reward + self.gamma * self.V[next_state[0], next_state[1]]
                        action_values.append(value)
                    
                    # Update value function
                    self.V[i, j] = max(action_values)
                    
                    # Track change
                    delta = max(delta, abs(v - self.V[i, j]))
            
            # Check convergence
            if delta < self.theta:
                print(f"Converged after {iteration + 1} iterations")
                break
        
        return self.V
    
    def extract_policy(self):
        """
        Extract optimal policy from value function
        """
        policy = np.zeros((self.env.size, self.env.size), dtype=int)
        
        for i in range(self.env.size):
            for j in range(self.env.size):
                if [i, j] == self.env.goal_pos:
                    continue
                
                # Find best action
                action_values = []
                for action in range(4):
                    self.env.agent_pos = [i, j]
                    next_state, reward, done = self.env.step(action)
                    value = reward + self.gamma * self.V[next_state[0], next_state[1]]
                    action_values.append(value)
                
                policy[i, j] = np.argmax(action_values)
        
        return policy
    
    def visualize(self, policy):
        """Visualize value function and policy"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Value function
        im1 = ax1.imshow(self.V, cmap='viridis')
        ax1.set_title('Value Function V(s)')
        plt.colorbar(im1, ax=ax1)
        
        # Add values as text
        for i in range(self.env.size):
            for j in range(self.env.size):
                ax1.text(j, i, f'{self.V[i, j]:.1f}',
                        ha='center', va='center', color='white')
        
        # Policy
        im2 = ax2.imshow(policy, cmap='tab10', vmin=0, vmax=3)
        ax2.set_title('Optimal Policy')
        plt.colorbar(im2, ax=ax2, ticks=[0, 1, 2, 3],
                    label='0=up, 1=down, 2=left, 3=right')
        
        # Add arrows
        arrows = ['↑', '↓', '←', '→']
        for i in range(self.env.size):
            for j in range(self.env.size):
                if [i, j] != self.env.goal_pos:
                    ax2.text(j, i, arrows[policy[i, j]],
                            ha='center', va='center', fontsize=20, color='white')
                else:
                    ax2.text(j, i, 'G', ha='center', va='center',
                            fontsize=20, color='red', weight='bold')
        
        plt.tight_layout()
        plt.show()

# Solve grid world with value iteration
env = SimpleGridWorld(size=4)
vi = ValueIteration(env, gamma=0.99)

print("Running Value Iteration...")
V = vi.solve()

print("\nExtracting optimal policy...")
policy = vi.extract_policy()

print("\nVisualization:")
vi.visualize(policy)

---

## 3. Classical RL: Q-Learning

### Q-Learning Algorithm

**Q-Learning** is a model-free, off-policy TD control algorithm.

**Update Rule**:

$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha [r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)]$$

where:
- $\alpha$: Learning rate
- $r_{t+1}$: Reward received
- $\gamma$: Discount factor
- TD Error: $\delta_t = r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)$

**Key Features**:
- **Model-free**: Doesn't need transition probabilities
- **Off-policy**: Learns optimal policy while following exploratory policy
- **Guaranteed convergence** (with appropriate learning rate schedule)

### Exploration vs Exploitation

**ε-greedy Policy**:
- With probability $\epsilon$: explore (random action)
- With probability $1-\epsilon$: exploit (best action)

$$a_t = \begin{cases}
\text{random action} & \text{with probability } \epsilon \\
\arg\max_a Q(s_t, a) & \text{with probability } 1-\epsilon
\end{cases}$$

In [ ]:
class QLearningAgent:
    """
    Q-Learning agent for discrete state-action spaces
    """
    
    def __init__(self, state_size, action_size, learning_rate=0.1,
                 gamma=0.99, epsilon=1.0, epsilon_decay=0.995,
                 epsilon_min=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Q-table: state -> action values
        self.Q = {}
    
    def get_q_value(self, state, action):
        """Get Q-value for state-action pair"""
        if state not in self.Q:
            self.Q[state] = np.zeros(self.action_size)
        return self.Q[state][action]
    
    def get_max_q_value(self, state):
        """Get maximum Q-value for state"""
        if state not in self.Q:
            self.Q[state] = np.zeros(self.action_size)
        return np.max(self.Q[state])
    
    def select_action(self, state, training=True):
        """
        Select action using ε-greedy policy
        """
        if training and np.random.random() < self.epsilon:
            # Explore: random action
            return np.random.randint(self.action_size)
        else:
            # Exploit: best action
            if state not in self.Q:
                self.Q[state] = np.zeros(self.action_size)
            return np.argmax(self.Q[state])
    
    def update(self, state, action, reward, next_state, done):
        """
        Q-Learning update
        """
        # Current Q-value
        current_q = self.get_q_value(state, action)
        
        # TD target
        if done:
            td_target = reward
        else:
            td_target = reward + self.gamma * self.get_max_q_value(next_state)
        
        # TD error
        td_error = td_target - current_q
        
        # Update Q-value
        if state not in self.Q:
            self.Q[state] = np.zeros(self.action_size)
        self.Q[state][action] += self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

def train_q_learning(env, agent, num_episodes=500):
    """
    Train Q-Learning agent
    """
    episode_rewards = []
    episode_lengths = []
    
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        steps = 0
        
        # Episode loop
        while steps < 100:  # Max steps per episode
            # Select action
            action = agent.select_action(state, training=True)
            
            # Take action
            next_state, reward, done = env.step(action)
            
            # Update Q-value
            agent.update(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Track metrics
        episode_rewards.append(total_reward)
        episode_lengths.append(steps)
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:])
            avg_length = np.mean(episode_lengths[-50:])
            print(f"Episode {episode + 1}/{num_episodes}, "
                  f"Avg Reward: {avg_reward:.2f}, "
                  f"Avg Length: {avg_length:.1f}, "
                  f"Epsilon: {agent.epsilon:.3f}")
    
    return episode_rewards, episode_lengths

# Train Q-Learning agent
env = SimpleGridWorld(size=4)
agent = QLearningAgent(state_size=(4, 4), action_size=4,
                      learning_rate=0.1, gamma=0.99,
                      epsilon=1.0, epsilon_decay=0.995)

print("Training Q-Learning agent...\n")
rewards, lengths = train_q_learning(env, agent, num_episodes=500)

# Plot learning curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Rewards
ax1.plot(rewards, alpha=0.3, label='Episode Reward')
ax1.plot(np.convolve(rewards, np.ones(50)/50, mode='valid'),
         label='Moving Average (50 episodes)', linewidth=2)
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('Q-Learning: Rewards over Time')
ax1.legend()
ax1.grid(True)

# Episode lengths
ax2.plot(lengths, alpha=0.3, label='Episode Length')
ax2.plot(np.convolve(lengths, np.ones(50)/50, mode='valid'),
         label='Moving Average (50 episodes)', linewidth=2)
ax2.set_xlabel('Episode')
ax2.set_ylabel('Steps to Goal')
ax2.set_title('Q-Learning: Episode Length over Time')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Test trained agent
print("\nTesting trained agent...")
state = env.reset()
env.render()

for step in range(20):
    action = agent.select_action(state, training=False)
    state, reward, done = env.step(action)
    print(f"Step {step + 1}: Action={action}, State={state}, Reward={reward}")
    
    if done:
        print("\nGoal reached!")
        env.render()
        break

---

## 4. Deep Q-Networks (DQN)

### From Q-Learning to Deep Q-Learning

**Problem with Q-Learning**: Cannot handle large/continuous state spaces (infinite table!)

**Solution**: Approximate Q-function with neural network:

$$Q(s, a; \theta) \approx Q^*(s, a)$$

### DQN Innovations

**1. Experience Replay**
- Store transitions $(s, a, r, s')$ in replay buffer
- Sample random minibatches for training
- Breaks correlation between consecutive samples

**2. Target Network**
- Use separate network for computing targets
- Update target network slowly (every C steps)
- Stabilizes learning

**Loss Function**:

$$L(\theta) = E_{(s,a,r,s') \sim D}\left[(r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta))^2\right]$$

where $\theta^-$ are target network parameters.

In [ ]:
class ReplayBuffer:
    """
    Experience Replay Buffer
    
    Stores transitions and samples random minibatches for training
    """
    
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
        self.Transition = namedtuple('Transition',
                                    ['state', 'action', 'reward', 'next_state', 'done'])
    
    def push(self, state, action, reward, next_state, done):
        """Add transition to buffer"""
        self.buffer.append(self.Transition(state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        """Sample random minibatch"""
        transitions = random.sample(self.buffer, batch_size)
        
        # Transpose batch
        batch = self.Transition(*zip(*transitions))
        
        # Convert to tensors
        states = torch.FloatTensor(np.array(batch.state))
        actions = torch.LongTensor(batch.action)
        rewards = torch.FloatTensor(batch.reward)
        next_states = torch.FloatTensor(np.array(batch.next_state))
        dones = torch.FloatTensor(batch.done)
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)

class DQN(nn.Module):
    """
    Deep Q-Network
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(DQN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, state):
        """Forward pass: state -> Q-values for all actions"""
        return self.network(state)

class DQNAgent:
    """
    DQN Agent with experience replay and target network
    """
    
    def __init__(self, state_dim, action_dim, learning_rate=1e-3,
                 gamma=0.99, epsilon=1.0, epsilon_decay=0.995,
                 epsilon_min=0.01, buffer_size=10000, batch_size=64,
                 target_update=10):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.target_update = target_update
        
        # Q-network and target network
        self.q_network = DQN(state_dim, action_dim)
        self.target_network = DQN(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()  # Target network in eval mode
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_size)
        
        # Training step counter
        self.steps = 0
    
    def select_action(self, state, training=True):
        """
        ε-greedy action selection
        """
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.action_dim)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                q_values = self.q_network(state_tensor)
                return q_values.argmax(dim=1).item()
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay buffer"""
        self.replay_buffer.push(state, action, reward, next_state, done)
    
    def update(self):
        """
        DQN update step
        """
        # Need enough samples in buffer
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Sample minibatch
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        
        # Compute current Q-values
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Compute target Q-values
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values
        
        # Compute loss
        loss = F.mse_loss(current_q_values, target_q_values)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), max_norm=1.0)
        self.optimizer.step()
        
        # Update target network
        self.steps += 1
        if self.steps % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()
    
    def decay_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

# Create CartPole environment
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"CartPole Environment:")
print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")
print(f"\nState: [cart position, cart velocity, pole angle, pole angular velocity]")
print(f"Actions: 0 = push left, 1 = push right")
print(f"\nGoal: Keep pole upright for as long as possible (max 500 steps)")

In [ ]:
def train_dqn(env, agent, num_episodes=500):
    """
    Train DQN agent
    """
    episode_rewards = []
    losses = []
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        total_reward = 0
        episode_losses = []
        
        # Episode loop
        for step in range(500):  # Max 500 steps
            # Select action
            action = agent.select_action(state, training=True)
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Store transition
            agent.store_transition(state, action, reward, next_state, done)
            
            # Update network
            loss = agent.update()
            if loss is not None:
                episode_losses.append(loss)
            
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Track metrics
        episode_rewards.append(total_reward)
        if episode_losses:
            losses.append(np.mean(episode_losses))
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:])
            print(f"Episode {episode + 1}/{num_episodes}, "
                  f"Avg Reward: {avg_reward:.1f}, "
                  f"Epsilon: {agent.epsilon:.3f}, "
                  f"Buffer Size: {len(agent.replay_buffer)}")
    
    return episode_rewards, losses

# Create and train DQN agent
agent = DQNAgent(state_dim=state_dim, action_dim=action_dim,
                learning_rate=1e-3, gamma=0.99,
                epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01,
                buffer_size=10000, batch_size=64, target_update=10)

print("\nTraining DQN agent on CartPole...\n")
rewards, losses = train_dqn(env, agent, num_episodes=500)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Rewards
ax1.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) >= 50:
    ax1.plot(np.convolve(rewards, np.ones(50)/50, mode='valid'),
             label='Moving Average (50 episodes)', linewidth=2)
ax1.axhline(y=195, color='r', linestyle='--', label='Solved Threshold (195)')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('DQN: Rewards over Time')
ax1.legend()
ax1.grid(True)

# Loss
if losses:
    ax2.plot(losses, alpha=0.6, label='Training Loss')
    if len(losses) >= 50:
        ax2.plot(np.convolve(losses, np.ones(50)/50, mode='valid'),
                 label='Moving Average (50 episodes)', linewidth=2)
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Loss')
    ax2.set_title('DQN: Training Loss over Time')
    ax2.legend()
    ax2.grid(True)

plt.tight_layout()
plt.show()

# Test trained agent
print("\nTesting trained DQN agent...")
test_rewards = []
for i in range(10):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(500):
        action = agent.select_action(state, training=False)
        state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        
        if terminated or truncated:
            break
    
    test_rewards.append(total_reward)
    print(f"Test Episode {i + 1}: Reward = {total_reward:.0f}")

print(f"\nAverage Test Reward: {np.mean(test_rewards):.1f} ± {np.std(test_rewards):.1f}")
print(f"CartPole is 'solved' when average reward > 195 over 100 episodes")

env.close()

---

## 5. Policy Gradient Methods

### From Value-Based to Policy-Based RL

**Value-based** (Q-Learning, DQN): Learn value function, derive policy

**Policy-based**: Directly learn policy $\pi_\theta(a|s)$

### REINFORCE Algorithm

**Objective**: Maximize expected return

$$J(\theta) = E_{\tau \sim \pi_\theta}[G_0] = E_{\tau \sim \pi_\theta}\left[\sum_{t=0}^T \gamma^t r_t\right]$$

**Policy Gradient Theorem**:

$$\nabla_\theta J(\theta) = E_{\tau \sim \pi_\theta}\left[\sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t|s_t) G_t\right]$$

**Interpretation**: 
- Increase probability of actions that lead to high returns
- Decrease probability of actions that lead to low returns

**Update Rule**:

$$\theta \leftarrow \theta + \alpha \nabla_\theta \log \pi_\theta(a_t|s_t) G_t$$

### Advantages

✅ Can handle continuous action spaces  
✅ Can learn stochastic policies  
✅ Better convergence properties  

### Disadvantages

❌ High variance  
❌ Sample inefficient  
❌ Can get stuck in local optima

In [ ]:
class PolicyNetwork(nn.Module):
    """
    Policy network for REINFORCE
    
    Outputs probability distribution over actions
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)  # Output probabilities
        )
    
    def forward(self, state):
        """Forward pass: state -> action probabilities"""
        return self.network(state)

class REINFORCEAgent:
    """
    REINFORCE (Monte Carlo Policy Gradient) agent
    """
    
    def __init__(self, state_dim, action_dim, learning_rate=1e-3, gamma=0.99):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        
        # Policy network
        self.policy = PolicyNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate)
        
        # Episode memory
        self.log_probs = []
        self.rewards = []
    
    def select_action(self, state):
        """
        Sample action from policy
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        action_probs = self.policy(state_tensor)
        
        # Sample action from distribution
        m = torch.distributions.Categorical(action_probs)
        action = m.sample()
        
        # Store log probability
        self.log_probs.append(m.log_prob(action))
        
        return action.item()
    
    def store_reward(self, reward):
        """Store reward"""
        self.rewards.append(reward)
    
    def update(self):
        """
        REINFORCE update at end of episode
        """
        # Compute returns (discounted cumulative rewards)
        returns = []
        G = 0
        for reward in reversed(self.rewards):
            G = reward + self.gamma * G
            returns.insert(0, G)
        
        # Normalize returns (variance reduction)
        returns = torch.FloatTensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)
        
        # Compute policy loss
        policy_loss = []
        for log_prob, G in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * G)
        
        policy_loss = torch.stack(policy_loss).sum()
        
        # Optimize
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        
        # Clear episode memory
        self.log_probs = []
        self.rewards = []
        
        return policy_loss.item()

def train_reinforce(env, agent, num_episodes=1000):
    """
    Train REINFORCE agent
    """
    episode_rewards = []
    losses = []
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        total_reward = 0
        
        # Collect episode
        for step in range(500):  # Max 500 steps
            # Select action
            action = agent.select_action(state)
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Store reward
            agent.store_reward(reward)
            
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        # Update policy at end of episode
        loss = agent.update()
        
        # Track metrics
        episode_rewards.append(total_reward)
        losses.append(loss)
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:])
            print(f"Episode {episode + 1}/{num_episodes}, "
                  f"Avg Reward: {avg_reward:.1f}")
    
    return episode_rewards, losses

# Create and train REINFORCE agent
env = gym.make('CartPole-v1')
agent = REINFORCEAgent(state_dim=state_dim, action_dim=action_dim,
                      learning_rate=1e-3, gamma=0.99)

print("Training REINFORCE agent on CartPole...\n")
rewards, losses = train_reinforce(env, agent, num_episodes=1000)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Rewards
ax1.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) >= 50:
    ax1.plot(np.convolve(rewards, np.ones(50)/50, mode='valid'),
             label='Moving Average (50 episodes)', linewidth=2)
ax1.axhline(y=195, color='r', linestyle='--', label='Solved Threshold (195)')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('REINFORCE: Rewards over Time')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(losses, alpha=0.3, label='Policy Loss')
if len(losses) >= 50:
    ax2.plot(np.convolve(losses, np.ones(50)/50, mode='valid'),
             label='Moving Average (50 episodes)', linewidth=2)
ax2.set_xlabel('Episode')
ax2.set_ylabel('Loss')
ax2.set_title('REINFORCE: Policy Loss over Time')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Test trained agent
print("\nTesting trained REINFORCE agent...")
test_rewards = []
for i in range(10):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(500):
        action = agent.select_action(state)
        state, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        
        if terminated or truncated:
            break
    
    # Clear episode memory
    agent.log_probs = []
    agent.rewards = []
    
    test_rewards.append(total_reward)
    print(f"Test Episode {i + 1}: Reward = {total_reward:.0f}")

print(f"\nAverage Test Reward: {np.mean(test_rewards):.1f} ± {np.std(test_rewards):.1f}")

env.close()

---

## 6. Actor-Critic Methods

### Combining Value-Based and Policy-Based RL

**Actor-Critic** = Policy Gradient + Value Function

**Actor** (Policy): $\pi_\theta(a|s)$ - decides which action to take

**Critic** (Value Function): $V_\phi(s)$ or $Q_\phi(s,a)$ - evaluates actions

### Advantage Actor-Critic (A2C)

**Advantage Function**:

$$A(s,a) = Q(s,a) - V(s)$$

Measures how much better action $a$ is compared to average.

**Policy Gradient with Advantage**:

$$\nabla_\theta J(\theta) = E[\nabla_\theta \log \pi_\theta(a|s) A(s,a)]$$

**TD-Error as Advantage**:

$$A(s,a) \approx \delta = r + \gamma V(s') - V(s)$$

### Benefits

✅ Lower variance than REINFORCE  
✅ More sample efficient  
✅ Online learning (update every step)

In [ ]:
class ActorCritic(nn.Module):
    """
    Actor-Critic network
    
    Shared feature extractor with separate heads for:
    - Actor: action probabilities
    - Critic: state value
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(ActorCritic, self).__init__()
        
        # Shared layers
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
        )
        
        # Actor head
        self.actor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)
        )
        
        # Critic head
        self.critic = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, state):
        """
        Forward pass
        
        Returns: (action_probs, state_value)
        """
        features = self.shared(state)
        action_probs = self.actor(features)
        state_value = self.critic(features)
        return action_probs, state_value

class A2CAgent:
    """
    Advantage Actor-Critic (A2C) agent
    """
    
    def __init__(self, state_dim, action_dim, learning_rate=1e-3, gamma=0.99):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        
        # Actor-Critic network
        self.ac_network = ActorCritic(state_dim, action_dim)
        self.optimizer = optim.Adam(self.ac_network.parameters(), lr=learning_rate)
    
    def select_action(self, state):
        """
        Select action from policy
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        action_probs, _ = self.ac_network(state_tensor)
        
        # Sample action
        m = torch.distributions.Categorical(action_probs)
        action = m.sample()
        
        return action.item()
    
    def update(self, state, action, reward, next_state, done):
        """
        A2C update (online, every step)
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
        
        # Get action probs and values
        action_probs, state_value = self.ac_network(state_tensor)
        _, next_state_value = self.ac_network(next_state_tensor)
        
        # Compute TD target and advantage
        if done:
            td_target = reward
        else:
            td_target = reward + self.gamma * next_state_value.item()
        
        advantage = td_target - state_value.item()
        
        # Actor loss (policy gradient with advantage)
        m = torch.distributions.Categorical(action_probs)
        log_prob = m.log_prob(torch.tensor(action))
        actor_loss = -log_prob * advantage
        
        # Critic loss (MSE between value and TD target)
        critic_loss = F.mse_loss(state_value, torch.tensor([[td_target]]))
        
        # Total loss
        loss = actor_loss + critic_loss
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()

def train_a2c(env, agent, num_episodes=1000):
    """
    Train A2C agent
    """
    episode_rewards = []
    losses = []
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        total_reward = 0
        episode_losses = []
        
        # Episode loop
        for step in range(500):  # Max 500 steps
            # Select action
            action = agent.select_action(state)
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Update (online)
            loss = agent.update(state, action, reward, next_state, done)
            episode_losses.append(loss)
            
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        # Track metrics
        episode_rewards.append(total_reward)
        losses.append(np.mean(episode_losses))
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:])
            print(f"Episode {episode + 1}/{num_episodes}, "
                  f"Avg Reward: {avg_reward:.1f}")
    
    return episode_rewards, losses

# Create and train A2C agent
env = gym.make('CartPole-v1')
agent = A2CAgent(state_dim=state_dim, action_dim=action_dim,
                learning_rate=1e-3, gamma=0.99)

print("Training A2C agent on CartPole...\n")
rewards, losses = train_a2c(env, agent, num_episodes=500)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Rewards
ax1.plot(rewards, alpha=0.3, label='Episode Reward')
if len(rewards) >= 50:
    ax1.plot(np.convolve(rewards, np.ones(50)/50, mode='valid'),
             label='Moving Average (50 episodes)', linewidth=2)
ax1.axhline(y=195, color='r', linestyle='--', label='Solved Threshold (195)')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('A2C: Rewards over Time')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(losses, alpha=0.3, label='Total Loss')
if len(losses) >= 50:
    ax2.plot(np.convolve(losses, np.ones(50)/50, mode='valid'),
             label='Moving Average (50 episodes)', linewidth=2)
ax2.set_xlabel('Episode')
ax2.set_ylabel('Loss')
ax2.set_title('A2C: Loss over Time')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

env.close()

---

## 7. Proximal Policy Optimization (PPO)

### The Challenge: Policy Gradient Instability

Problem: Large policy updates can cause performance collapse

### PPO Solution: Clipped Surrogate Objective

**Probability Ratio**:

$$r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$$

**Clipped Objective**:

$$L^{CLIP}(\theta) = E_t[\min(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t)]$$

where $\epsilon$ is typically 0.2.

**Interpretation**:
- If advantage is positive: clip ratio to $[1, 1+\epsilon]$ (don't increase prob too much)
- If advantage is negative: clip ratio to $[1-\epsilon, 1]$ (don't decrease prob too much)

### Why PPO is Popular

✅ Stable training  
✅ Sample efficient  
✅ Works well with continuous actions  
✅ State-of-the-art performance  
✅ Easy to implement  

**Used in**: OpenAI Five (Dota 2), DeepMind's AlphaStar (StarCraft II)

---

## 8. Advanced Topics

### 8.1 Multi-Armed Bandits

**Simplest RL problem**: No states, just actions and rewards

**Exploration Strategies**:
- ε-greedy
- Upper Confidence Bound (UCB)
- Thompson Sampling

### 8.2 Model-Based RL

**Learn environment model**: $P(s'|s,a)$ and $R(s,a)$

**Then plan** using the model

**Advantages**:
- More sample efficient
- Can plan ahead

### 8.3 Offline RL

Learn from fixed dataset (no environment interaction)

**Applications**: Healthcare, robotics, autonomous driving

### 8.4 Multi-Agent RL

Multiple agents learning simultaneously

**Challenges**:
- Non-stationary environment
- Credit assignment
- Communication

### 8.5 Hierarchical RL

Learn hierarchical policies (high-level goals → low-level actions)

**Approaches**:
- Options framework
- Feudal RL
- HIRO

### 8.6 Inverse RL

Learn reward function from expert demonstrations

**Applications**: Imitation learning, apprenticeship learning

---

## 9. Interview Questions

### Fundamentals

**Q1: What is the difference between supervised learning and reinforcement learning?**

**A**: 
- **Supervised Learning**: Learn from labeled examples $(x, y)$. Goal: minimize prediction error.
- **Reinforcement Learning**: Learn from rewards through trial and error. Goal: maximize cumulative reward.

Key differences:
- SL: Direct feedback (correct answer)
- RL: Delayed feedback (reward after sequence of actions)
- SL: i.i.d. data
- RL: Sequential, correlated data

---

**Q2: Explain the exploration-exploitation tradeoff.**

**A**: 
- **Exploitation**: Use current knowledge to maximize reward (greedy)
- **Exploration**: Try new actions to gain more information

Too much exploitation: May miss better options  
Too much exploration: Waste time on suboptimal actions

Solutions: ε-greedy, UCB, Thompson Sampling

---

**Q3: What is the discount factor γ and why is it important?**

**A**: Discount factor $\gamma \in [0, 1]$ determines how much we value future rewards:

$$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ...$$

- $\gamma = 0$: Only care about immediate reward (myopic)
- $\gamma = 1$: All future rewards equally important
- $\gamma = 0.99$ (typical): Future rewards slightly discounted

Why discount?
1. Uncertainty increases with time
2. Mathematical convergence
3. Finite horizon episodes

---

**Q4: What is the Bellman equation?**

**A**: Recursive relationship for value functions:

$$V^\pi(s) = \sum_a \pi(a|s) \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^\pi(s')]$$

Interpretation: Value of state = Expected immediate reward + Discounted value of next state

Optimality version:

$$V^*(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^*(s')]$$

---

### Algorithms

**Q5: Compare Q-Learning and SARSA.**

**A**:

**Q-Learning (off-policy)**:
- Update: $Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$
- Learns optimal policy while following exploratory policy
- More aggressive, can diverge

**SARSA (on-policy)**:
- Update: $Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma Q(s',a') - Q(s,a)]$
- Learns policy it's actually following
- More conservative, safer

---

**Q6: Why does DQN use experience replay and target networks?**

**A**:

**Experience Replay**:
- **Problem**: Consecutive samples are highly correlated
- **Solution**: Store transitions, sample random minibatches
- **Benefits**: Breaks correlation, reuses data (sample efficient)

**Target Network**:
- **Problem**: Chasing moving target (Q-values change during training)
- **Solution**: Separate network for targets, update slowly
- **Benefits**: Stabilizes training, prevents oscillations

---

**Q7: What is the advantage function and why is it useful?**

**A**: 

$$A(s,a) = Q(s,a) - V(s)$$

Measures: How much better is action $a$ compared to average?

**Benefits**:
- Reduces variance in policy gradients
- Centers rewards around zero
- Only updates based on relative advantage

Used in: A2C, PPO, GAE (Generalized Advantage Estimation)

---

**Q8: Compare value-based and policy-based methods.**

**A**:

**Value-Based (Q-Learning, DQN)**:
- Learn: Value function $Q(s,a)$
- Policy: $\pi(s) = \arg\max_a Q(s,a)$
- Pros: Sample efficient, stable
- Cons: Can't handle continuous actions, deterministic policy

**Policy-Based (REINFORCE, PPO)**:
- Learn: Policy $\pi_\theta(a|s)$ directly
- Pros: Continuous actions, stochastic policies
- Cons: High variance, sample inefficient

**Actor-Critic**: Best of both worlds!

---

### Advanced

**Q9: How would you handle continuous action spaces?**

**A**: Several approaches:

1. **Policy Gradient Methods** (PPO, DDPG):
   - Output Gaussian distribution: $\pi_\theta(a|s) = N(\mu_\theta(s), \sigma_\theta(s))$
   - Sample from distribution

2. **Discretization**:
   - Bin continuous space into discrete actions
   - Simple but loses precision

3. **Action Parameterization** (DDPG, TD3, SAC):
   - Actor outputs continuous action directly
   - Critic evaluates state-action pairs

---

**Q10: What are the main challenges in deep RL?**

**A**:

1. **Sample Efficiency**: Need millions of samples (expensive in real world)
2. **Stability**: Non-stationary targets, moving distributions
3. **Credit Assignment**: Which action caused the reward?
4. **Exploration**: How to explore efficiently in large spaces?
5. **Sparse Rewards**: Long sequences before reward
6. **Partial Observability**: Don't see full state
7. **Reproducibility**: Sensitive to hyperparameters, random seeds

**Solutions**:
- Experience replay, target networks (stability)
- Curiosity-driven exploration
- Hindsight Experience Replay (sparse rewards)
- Recurrent networks (partial observability)
- Model-based RL (sample efficiency)

---

## Summary

**We covered**:

1. **RL Fundamentals**: MDPs, value functions, policies
2. **Classical RL**: Q-Learning, value iteration
3. **Deep RL**: DQN with experience replay and target networks
4. **Policy Gradients**: REINFORCE algorithm
5. **Actor-Critic**: A2C combining value and policy
6. **Advanced**: PPO, multi-agent RL, model-based RL

**Key Takeaways**:
- RL learns from trial and error to maximize cumulative reward
- Exploration vs exploitation is fundamental
- Value-based methods (DQN) vs Policy-based (REINFORCE) vs Actor-Critic (best of both)
- Deep RL enables solving complex tasks (games, robotics, etc.)
- Still active research area with many open challenges

**Further Learning**:
- Sutton & Barto: "Reinforcement Learning: An Introduction"
- OpenAI Spinning Up: https://spinningup.openai.com/
- DeepMind x UCL RL Course

---

**References**:
1. Sutton & Barto, "Reinforcement Learning: An Introduction" (2018)
2. Mnih et al., "Playing Atari with Deep Reinforcement Learning" (2013)
3. Mnih et al., "Human-level control through deep reinforcement learning" (2015)
4. Schulman et al., "Proximal Policy Optimization Algorithms" (2017)
5. Lillicrap et al., "Continuous control with deep reinforcement learning" (2015)

---

*Created: October 2025*  
*Last Updated: October 2025*